In [2]:
import cv2
import toml
import sys
import os
import time
import json
import numpy as np
from IPython.display import display, Image, clear_output

import pynq

sys.path.append(os.path.abspath("data_management"))
sys.path.append(os.path.abspath("onnx_models"))
sys.path.append(os.path.abspath("hardware"))

import data_preprocessing as dprep
import data_postprocessing as dpostp
import onnx_inference as onnx_inf
import hw_partition as hw_part

%matplotlib inline
!sudo usermod -a -G video root

In [3]:
yolo_cfg = toml.load('yolov5n.toml')
inf_exec = yolo_cfg['inf_exec']
imgsz = yolo_cfg['input_data']['imgsz']
conf_thres = yolo_cfg['predictor']['conf_thres']
iou_thres = yolo_cfg['predictor']['iou_thres']
classes = yolo_cfg['predictor']['classes']
if not classes:
    classes = None
onnx_model_path = yolo_cfg['onnx_model_path']
if inf_exec == "fpga":
    onnx_model_path = f"{onnx_model_path.split('.onnx')[0]}_head.onnx"
visualize = yolo_cfg['visualize']
out_img_path = yolo_cfg['out_img_path']
input_source = yolo_cfg['source']
bitstream_path = yolo_cfg['hardware']['bitstream_path']
weights_lookup_table = yolo_cfg['hardware']['weights_lookup_table']
weights_path = yolo_cfg['hardware']['weights_path']
input_bp = yolo_cfg['hardware']['input_binary_point']
output_bp = yolo_cfg['hardware']['output_binary_point']
fifo_depth = yolo_cfg['hardware']['fifo_depth']

In [4]:
stride, names, session, output_names = onnx_inf.load_model(onnx_model_path)

In [5]:
#########################
### fpgaConvNet Setup ###
#########################

# initialise partition
partition = hw_part.Partition(bitstream_path, 5) 

# add input buffers
partition.add_input_buffer(0, 0, [320, 320, 3], bp=input_bp)

# add output buffers
partition.add_output_buffer(2, 2, [40, 40, 256], bp=output_bp, streams=2)
partition.add_output_buffer(3, 3, [20, 20, 256], bp=output_bp, streams=2)
partition.add_output_buffer(4, 4, [10, 10, 256], bp=output_bp, streams=2)

# create fifos
partition.add_fifo(0, 0, 2, 40*40*64 , burst=fifo_depth[0])
partition.add_fifo(1, 1, 1, 20*20*128, burst=fifo_depth[1])

# # setup hardware
partition.reset_hardware()

# get the lookup table for the weights
with open(weights_lookup_table, "r") as f:
    lookup = json.load(f)

# iterate over the weights
for layer, idx in lookup.items():

    # allocate weights and load them
    start_time = time.perf_counter() 
    partition.reload_weights(idx, f"{weights_path}/{layer}.dat")
    pred_time = (time.perf_counter() - start_time)*1000
    print(f"[{idx}] {layer} loaded! ({pred_time:.2f} ms)")

# setup hardware
partition.reset_hardware()
partition.start_hardware() 

[0] conv_0 loaded! (27.60 ms)
[1] conv_2 loaded! (37.55 ms)
[2] conv_4 loaded! (18.08 ms)
[3] conv_11 loaded! (21.85 ms)
[4] conv_6 loaded! (21.01 ms)
[5] conv_8 loaded! (24.22 ms)
[6] conv_14 loaded! (19.56 ms)
[7] conv_16 loaded! (93.06 ms)
[8] conv_18 loaded! (23.11 ms)
[9] conv_30 loaded! (27.19 ms)
[10] conv_20 loaded! (23.74 ms)
[11] conv_22 loaded! (51.82 ms)
[12] conv_25 loaded! (19.66 ms)
[13] conv_27 loaded! (51.57 ms)
[14] conv_33 loaded! (32.96 ms)
[15] conv_35 loaded! (639.26 ms)
[16] conv_37 loaded! (48.51 ms)
[17] conv_54 loaded! (49.06 ms)
[18] conv_39 loaded! (34.48 ms)
[19] conv_41 loaded! (146.17 ms)
[20] conv_44 loaded! (34.08 ms)
[21] conv_46 loaded! (141.44 ms)
[22] conv_49 loaded! (34.31 ms)
[23] conv_51 loaded! (141.35 ms)
[24] conv_57 loaded! (78.19 ms)
[25] conv_59 loaded! (935.32 ms)
[26] conv_61 loaded! (128.87 ms)
[27] conv_68 loaded! (136.35 ms)
[28] conv_63 loaded! (76.28 ms)
[29] conv_65 loaded! (491.02 ms)
[30] conv_71 loaded! (234.80 ms)
[31] conv_73 l

In [12]:
from pynq import get_rails, DataRecorder

rails = get_rails()
recorder = DataRecorder(rails['12V'].power)



In [15]:
if input_source == 'webcam':
    cap = cv2.VideoCapture(0) 
else:
    cap = cv2.VideoCapture(input_source)

# Capture frame-by-frame
ret, frame = cap.read()
    
frame_count = 0

# Capture frame-by-frame
ret, frame = cap.read()

# Check that frame is not empty
orig_img = frame.copy()

img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
img = dprep.img_preprocess(img, imgsz, stride)
            
# assign the input data
data = np.moveaxis(img[0], 0, -1).flatten() * (2**partition.input_bp[0])
partition.input_buffers[0][:] = data.astype(np.int16)

# flush the buffers
partition.input_buffers[0].flush()
partition.output_buffers[2].flush()
partition.output_buffers[3].flush()
partition.output_buffers[4].flush()

for i in range(len(partition.fifo_buffers[0])):
    partition.fifo_buffers[0][i].flush()
for i in range(len(partition.fifo_buffers[1])):
    partition.fifo_buffers[1][i].flush()

# start all transfers
start_time = time.perf_counter()  

# start recording power 
with recorder.record(0.001): # Sample every 1 ms (if possible)

    results = recorder.frame
    # recieve data out
    partition.recv_dma(2)
    partition.recv_dma(3)
    partition.recv_dma(4)

    # start transferring data in
    partition.send_dma(0)  

    # start the FIFOs
    partition.start_fifo([0, 1])

    # wait for DMAs to finish
    partition.wait_dma(0, recv=False)
    partition.wait_dma(2, send=False)
    partition.wait_dma(3, send=False)
    partition.wait_dma(4, send=False)

# print statement
pred_time = (time.perf_counter() - start_time)*1000

# reset the FIFO
partition.reset_fifo_dma()

# print the power summary
print(recorder.frame)

                            Mark  12V_power
2022-10-30 08:49:57.767169   0.0     14.862
2022-10-30 08:49:57.786111   0.0     15.137
2022-10-30 08:49:57.797706   0.0     15.500
2022-10-30 08:49:57.808865   0.0     14.600
2022-10-30 08:49:57.821370   0.0     15.112
2022-10-30 08:50:05.170916   1.0     14.875
2022-10-30 08:50:05.183554   1.0     15.387
2022-10-30 08:50:05.195542   1.0     14.550
2022-10-30 08:50:05.206827   1.0     15.225
2022-10-30 08:50:05.219433   1.0     15.025
2022-10-30 08:50:05.231545   1.0     14.712


In [16]:
try:
    cap.release()
    cap.destroyAllWindows()
except Exception as e:
    print(e)
    pass

'cv2.VideoCapture' object has no attribute 'destroyAllWindows'
